In [1]:
import pandas as pd
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import webbrowser
import threading

# Read the SpaceX launch data from the correct path
spacex_df = pd.read_csv(r"C:\Users\allco\UDLA_MSc\spacex_launch_dash.csv")

# Find the min and max values of the payload for slider setup
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a Dash application
app = Dash(__name__)

# Layout of the dashboard with full screen style
app.layout = html.Div(
    children=[
        html.H1(
            'SpaceX Launch Records Dashboard', 
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}
        ),
        
        # Dropdown to select launch site
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
            ],
            value='ALL',  # Default to show all sites
            placeholder="Select a Launch Site here",
            searchable=True
        ),
        
        html.Br(),

        # Pie chart for launch success
        html.Div(dcc.Graph(id='success-pie-chart')),
        
        html.Br(),

        # Range slider for selecting payload range
        html.P("Payload range (Kg):"),
        dcc.RangeSlider(
            id='payload-slider',
            min=0, 
            max=10000, 
            step=1000,
            marks={0: '0', 10000: '10000'},
            value=[min_payload, max_payload]
        ),
        
        # Scatter plot for success vs payload
        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ],
    style={
        'height': '100vh',  # Full height for the entire page
        'margin': '0',  # Remove any default margin
        'padding': '0',  # Remove any default padding
        'overflow': 'hidden'  # Prevent scrollbars
    }
)

# Callback function to update the pie chart based on the selected site
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, names='Launch Site', 
                     title='Total Launch Success for All Sites')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, names='class', 
                     title=f'Success vs Failure for {entered_site}')
        return fig

# Callback function to update the scatter chart based on the selected site and payload range
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload-slider", component_property="value")]
)
def get_scatter_chart(entered_site, payload_range):
    filtered_df = spacex_df
    min_payload, max_payload = payload_range
    filtered_df = filtered_df[filtered_df['Payload Mass (kg)'].between(min_payload, max_payload)]
    
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
    
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
                     color='Booster Version Category', 
                     title=f"Payload vs. Success for {entered_site if entered_site != 'ALL' else 'All Sites'}")
    return fig

# Function to automatically open the app in a new browser tab
def open_browser():
    webbrowser.open("http://127.0.0.1:8050")

# Run the app in a new thread and open the browser automatically
if __name__ == '__main__':
    threading.Timer(1, open_browser).start()  # Wait 1 second before opening the browser
    app.run_server(debug=True, use_reloader=False)  # use_reloader=False to avoid re-running the script

